## Exercise: Clipping an array

Yes, NumPy has a `clip` ufunc already, but let's pretend it doesn't.  

Create a Numba vectorized ufunc that takes a vector `a`, a lower limit `amin` and an upper limit `amax`.  It should return the vector `a` with all values clipped such that $a_{min} < a < a_{max}$


In [2]:
from numba import vectorize

In [10]:
def clip(a, amin,amax):
    if amin > a:
        a = amin
    elif amax < a:
        a = amax
    return(a)
        


First, check the function works

In [14]:
clip(4.5,4.4,4.41)

4.41

But we cannot apply `clip` to arrays

In [15]:
import numpy as np

In [23]:
clip(np.random.random(20),0.2,0.8)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

We simply feed the function to `numba.vectorize`, and we get an optimized vectorized function

In [18]:
from numba import vectorize

In [19]:
vec_clip = vectorize()(clip)

In [22]:
vec_clip(np.random.random(20),0.2,0.8)

array([ 0.8       ,  0.2       ,  0.59073545,  0.8       ,  0.38690595,
        0.33073803,  0.8       ,  0.2411705 ,  0.8       ,  0.61059995,
        0.8       ,  0.24063495,  0.2       ,  0.42573273,  0.23056033,
        0.8       ,  0.24086422,  0.28745967,  0.72981381,  0.2       ])

## Exercise: Create `logit` ufunc

Recall from above that this is a ufunc which performs this operation:

$$f(a) = \log \left(\frac{a}{1-a}\right)$$

In [24]:
from math import log

`math.log` is $log_e$, ie $ln$

In [26]:
log(np.e)

1.0

In [27]:
def logit(a):
    return(log(a) - log(1-a))

In [28]:
vec_logit = vectorize()(logit)

The domain of $\log \left(\frac{a}{1-a}\right)$ is $a \in (-1,1)$

In [38]:
n = 1000
np.random.seed(0)
%timeit vec_logit(np.random.random(n))

29.9 µs ± 324 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [35]:
def loop_logit(a_arr):
    for i,a in enumerate(a_arr):
        a_arr[i] = logit(a)
    return(a_arr)
        

In [39]:
n = 1000
np.random.seed(0)
%timeit loop_logit(np.random.random(n))

740 µs ± 6.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


So we are getting a 25x speedup using `numba.vectorize`, with just a few lines of code, over naive looping. The `loop_logit` function actually has more extra code (`for i,a in enumerate(a_arr):`) than `vectorize()`